In [1]:
import torch
import numpy as np
import Data_Generator
import torch.nn as nn

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.get_device_name(0)

'GeForce RTX 2060 SUPER'

In [4]:
from Data_Generator import TSPDataset

In [5]:
dataset = TSPDataset(10000, 5)


Data points 2987/10000:  30%|██████████████▌                                  | 2960/10000 [00:01<00:02, 2346.95data/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Data points 8480/10000:  85%|█████████████████████████████████████████▌       | 8479/10000 [00:03<00:00, 2361.49data/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Solved 4065/10000:  40%|█████████████████████                                | 3983/10000 [00:02<00:04, 1440.09s

In [6]:
points_list = dataset.data['Points_List']
solutions = dataset.data['Solutions']

In [21]:
transition_matrix = np.zeros((len(first_cities), len(first_cities)))

In [22]:
transition_matrix

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [78]:
def route_distance(cities):
    distance = 0
    for i in range(1, len(cities)):
        a = cities[i -1]
        b = cities[i]
        distance += np.linalg.norm(a-b)
    a = cities[len(cities)-1]
    b = cities[0]
    distance += np.linalg.norm(a-b)
    return distance

In [24]:
class ActorCritic(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, learning_rate=3e-4):
        super(ActorCritic, self).__init__()

        self.num_actions = num_actions
        self.critic_linear1 = nn.Linear(num_inputs, hidden_size)
        self.critic_linear2 = nn.Linear(hidden_size, 1)

        self.actor_linear1 = nn.Linear(num_inputs, hidden_size)
        self.actor_linear2 = nn.Linear(hidden_size, num_actions)
        self.saved_actions = []
        self.rewards = []
    
    def forward(self, state):
        state = Variable(torch.from_numpy(state).float().unsqueeze(0))
        value = F.relu(self.critic_linear1(state))
        value = self.critic_linear2(value)
        
        policy_dist = F.relu(self.actor_linear1(state))
        policy_dist = F.softmax(self.actor_linear2(policy_dist), dim=1)

        return value, policy_dist

In [27]:
from matplotlib import pyplot as plt

In [28]:
first_cities = points_list[0]
first_solutions = solutions[0]

In [30]:
transition_matrix[0]

array([0., 0., 0., 0., 0.])

In [68]:
def select_action(state, already_taken_actions):
    probabilities = transition_matrix[state]
    sorted_inidices = np.argsort(probabilities)
    for i in sorted_inidices:
        if i not in already_taken_actions:
            return i
    print ("Sth wen wrong")    

In [104]:
GAMMA  = 0.005

In [105]:
def sample_episodes(start_state):
    cities_number = 0
    current_state = start_state
    tour = [current_state]
    while cities_number < len(first_cities) -1:
        current_state = select_action(current_state, tour)
        tour.append(current_state)
        cities_number +=1
    return tour
        

In [106]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


In [159]:
actor_critic =  ActorCritic(1, 5, 4)

In [160]:
baseline = 0
import torch.optim as optim


In [161]:
ac_optimizer = optim.Adam(actor_critic.parameters(), lr=0.01)

In [165]:
for t in range(1000):
    samples = []
    for i in range(len(first_cities)):
        samples.append(sample_episodes(i))
    lengths = [route_distance(first_cities[cities]) for cities in samples]
    j = np.argmin(lengths)
    reward = -lengths[j]
    shortest_tour = samples[j]
    
    # update actor critic
    log_probs = torch.log(torch.from_numpy(transition_matrix[:j]))

    advantage = reward - baseline
    actor_loss = (log_probs * advantage).mean()
    critic_loss = advantage.pow(2).mean()
    ac_loss = actor_loss + critic_loss 
   
    ac_optimizer.zero_grad()
    ac_loss.requires_grad=True
    ac_loss.backward()
    ac_optimizer.step()
    
    baseline, probab = actor_critic.forward(np.array([j]))

RuntimeError: you can only change requires_grad flags of leaf variables.

In [ ]:
num_inputs = 0
num_outputs = len(first_cities)

actor_critic = ActorCritic(num_inputs, num_outputs, hidden_size)
ac_optimizer = optim.Adam(actor_critic.parameters(), lr=learning_rate)

all_lengths = []
average_lengths = []
all_rewards = []
entropy_term = 0

for episode in range(max_episodes):
    log_probs = []
    values = []
    rewards = []

    state = 0
    for steps in range(num_steps):
        
        value, policy_dist = actor_critic.forward(state)
        value = value.detach().numpy()[0, 0]
        dist = policy_dist.detach().numpy()

        action = np.random.choice(num_outputs, p=np.squeeze(dist))
        log_prob = torch.log(policy_dist.squeeze(0)[action])
        entropy = -np.sum(np.mean(dist) * np.log(dist))
        new_state, reward, done, _ = env.step(action)

        rewards.append(reward)
        values.append(value)
        log_probs.append(log_prob)
        entropy_term += entropy
        state = new_state

        if done or steps == num_steps - 1:
            Qval, _ = actor_critic.forward(new_state)
            Qval = Qval.detach().numpy()[0, 0]
            all_rewards.append(np.sum(rewards))
            all_lengths.append(steps)
            average_lengths.append(np.mean(all_lengths[-10:]))
            if episode % 10 == 0:
                sys.stdout.write(
                    "episode: {}, reward: {}, total length: {}, average length: {} \n".format(episode, np.sum(rewards),
                                                                                              steps,
                                                                                              average_lengths[-1]))
            break

    # compute Q values
    Qvals = np.zeros_like(values)
    for t in reversed(range(len(rewards))):
        Qval = rewards[t] + GAMMA * Qval
        Qvals[t] = Qval

    # update actor critic
    values = torch.FloatTensor(values)
    Qvals = torch.FloatTensor(Qvals)
    log_probs = torch.stack(log_probs)

    advantage = Qvals - values
    actor_loss = (-log_probs * advantage).mean()
    critic_loss = 0.5 * advantage.pow(2).mean()
    ac_loss = actor_loss + critic_loss + 0.001 * entropy_term
   
    ac_optimizer.zero_grad()
    ac_loss.backward()
    ac_optimizer.step()

# Plot results
smoothed_rewards = pd.Series.rolling(pd.Series(all_rewards), 10).mean()
smoothed_rewards = [elem for elem in smoothed_rewards]
plt.plot(all_rewards)
plt.plot(smoothend_rewards)
plt.plot()
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.show()

plt.plot(all_lengths)
plt.plot(average_lengths)
plt.xlabel('Episode')
plt.ylabel('Episode length')
plt.show()
